##Introduction to Prompt Engineering
##Restaurant Review Analysis
## Problem Statement


The company receives large volumes of customer reviews across its restaurants. These reviews contain valuable insights that can guide business and marketing decisions.

## Problem Definition
Manual analysis of unstructured text reviews is slow and unscalable.
The company struggles to automatically extract and understand customer sentiments (positive, negative, or neutral) from this data.

## Objective
Build an automated sentiment analysis model using an LLM to predict customer sentiment, enabling data-driven insights and improved customer satisfaction.

## Installing and Importing Necessary Libraries


In [ ]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.45 --force-reinstall --no-cache-dir -q

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.45 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/36.7 MB 101.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 315.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 259.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 446.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 221.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 330.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for llama-cpp-python
ERROR: ERROR: Failed to buil

In [ ]:
# For downloading the models from HF Hub
!pip install huggingface_hub
!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 29.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4422719 sha256=ec1e0d409db28720350d864227a2c8287b80c8b9b187e1749047c0590024e8aa
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python


In [ ]:
# Importing library for data manipulation
import pandas as pd

# Function to download the model from the Hugging Face model hub
from huggingface_hub import hf_hub_download

# Importing the Llama class from the llama_cpp module
from llama_cpp import Llama

# Importing the json module
import json

In [ ]:
data = pd.read_csv("/content/restaurant_reviews.csv")
data['review_full'][5]

"We came across Perch by accident and had dinner there on two separate occasions, having enjoyed it so much the first time round. There is no doubt that it has a strong European emphasis, although results in fantastic, modern food and attentive service that was amongst the best that we've experienced here.  The Indian Sauvignon Blanc was very acceptable, rather than the more expensive European alternatives, and there was a good range of cocktails, mocktails and beers. Decor was stylish but simple, ambience was good with lively, tasteful music to suit the meal. All food we ordered was good, and we've taken a menu home to replicate some of our favourite dishes. Have two appetisers or one main each, a couple of puddings are good to share on a small table.  Half carafe of wine, half of Sangria, two beers, five appetisers and two puddings including taxes and service about £75. "

In [ ]:
# checking the first five rows of the data
data.head()

,restaurant_ID,rating_review,review_full
0,FLV202,5,"Totally in love with the Auro of the place, re..."
1,SAV303,5,Kailash colony is brimming with small cafes no...
2,YUM789,5,Excellent taste and awesome decorum. Must visi...
3,TST101,5,I have visited at jw lough/restourant. There w...
4,EAT456,5,Had a great experience in the restaurant food ...


In [ ]:
data['review_full'][3]

'I have visited at jw lough/restourant. There were a first class service at lough, specially Ms.laxmi  who were superbed for handling the client need, me and my family lots enjoyed her specialty in the manner, and Laxmi is a very very good in the client service, I hope when I will come against I would definitely serve from Ms. Laxmi and she is wonderful girl in that service. See you again Ms. Laxmi for the your best service which I have received from you at jw lough/resourant. Thank you JW Marriott Hotel at Atrocity, Delhi'

In [ ]:
# checking the shape of the data (20 rows 3 columns)
data.shape

(20, 3)

In [ ]:
# checking for missing values
##There are no missing values in the data
data.isnull().sum()

,0
restaurant_ID,0
rating_review,0
review_full,0


## Model Building


In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf" # the model is in gguf format

In [ ]:
# Using hf_hub_download to download a model from the Hugging Face model hub
# The repo_id parameter specifies the model name or path in the Hugging Face repository
# The filename parameter specifies the name of the file to download
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [ ]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # CPU cores
    n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    # n_gpu_layers=43,  # uncomment and change this value based on GPU VRAM pool.
    n_ctx=4096,  # Context window
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

In [ ]:
llm = Llama(
    model_path=model_path,
    n_ctx=1024,
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loade

## Defining Model Response Parameters

In [ ]:
def generate_llama_response(instruction, review):

    # System message explicitly instructing not to include the review text
    system_message = """
        [INST]<<SYS>>
        {}
        <</SYS>>[/INST]
    """.format(instruction)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"{review}\n{system_message}"

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1024,
        temperature=0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False,
        seed=42,
    )

    # Extract the sentiment from the response
    response_text = response["choices"][0]["text"]
    return response_text

- **`max_tokens`**: This parameter **specifies the maximum number of tokens that the model should generate** in response to the prompt.

- **`temperature`**: This parameter **controls the randomness of the generated response**. A higher temperature value will result in a more random response, while a lower temperature value will result in a more predictable response.

- **`top_p`**: This parameter **controls the diversity of the generated response by establishing a cumulative probability cutoff for token selection**. A higher value of top_p will result in a more diverse response, while a lower value will result in a less diverse response.

- **`repeat_penalty`**: This parameter **controls the penalty for repeating tokens in the generated response**. A higher value of repeat_penalty will result in a lower probability of repeating tokens, while a lower value will result in a higher probability of repeating tokens.

- **`top_k`**: This parameter **controls the maximum number of most-likely next tokens to consider** when generating the response at each step.

- **`stop`**: This parameter is a **list of tokens that are used to dynamically stop response generation** whenever the tokens in the list are encountered.

- **`echo`**: This parameter **controls whether the input (prompt) to the model should be returned** in the model response.

- **`seed`**: This parameter **specifies a seed value that helps replicate results**.


## Utility function

In [ ]:
# defining a function to parse the JSON output from the model
def extract_json_data(json_str):
    try:
        # Find the indices of the opening and closing curly braces
        json_start = json_str.find('{')
        json_end = json_str.rfind('}')

        if json_start != -1 and json_end != -1:
            extracted_sentiment = json_str[json_start:json_end + 1]  # Extract the JSON object
            data_dict = json.loads(extracted_sentiment)
            return data_dict
        else:
            print(f"Warning: JSON object not found in response: {json_str}")
            return {}
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return {}

## 1. Sentiment Analysis (Llama)

In [ ]:
# creating a copy of the data
data_1 = data.copy()

In [ ]:
# defining the instructions for the model
instruction_1 = """
    You are an AI analyzing restaurant reviews. Classify the sentiment of the provided review into the following categories:
    - Positive
    - Negative
    - Neutral
"""

In [ ]:
data_1['model_response'] = data_1['review_full'].apply(lambda x: generate_llama_response(instruction_1, x))

llama_perf_context_print:        load time =   78131.94 ms
llama_perf_context_print: prompt eval time =   78131.68 ms /   237 tokens (  329.67 ms per token,     3.03 tokens per second)
llama_perf_context_print:        eval time =   85017.49 ms /   215 runs   (  395.43 ms per token,     2.53 tokens per second)
llama_perf_context_print:       total time =  163272.43 ms /   452 tokens
llama_perf_context_print:    graphs reused =        207
Llama.generate: 1 prefix-match hit, remaining 307 prompt tokens to eval
llama_perf_context_print:        load time =   78131.94 ms
llama_perf_context_print: prompt eval time =   98751.38 ms /   307 tokens (  321.67 ms per token,     3.11 tokens per second)
llama_perf_context_print:        eval time =   63654.07 ms /   159 runs   (  400.34 ms per token,     2.50 tokens per second)
llama_perf_context_print:       total time =  162490.00 ms /   466 tokens
llama_perf_context_print:    graphs reused =        153
Llama.generate: 1 prefix-match hit, remaining 

KeyboardInterrupt: 

In [ ]:
data_1['model_response'].head()

In [ ]:
data.shape

In [ ]:
i = 2
print(data_1.loc[i, 'review_full'])

In [ ]:
print(data_1.loc[i, 'model_response'])

In [ ]:
def extract_sentiment(model_response):
    if 'positive' in model_response.lower():
        return 'Positive'
    elif 'negative' in model_response.lower():
        return 'Negative'
    elif 'neutral' in model_response.lower():
        return 'Neutral'

In [ ]:
# applying the function to the model response
data_1['sentiment'] = data_1['model_response'].apply(extract_sentiment)
data_1['sentiment'].head()

In [ ]:
data_1['sentiment'].value_counts()

In [ ]:
final_data_1 = data_1.drop(['model_response'], axis=1)
final_data_1.head()

## 1. Sentiment Analysis (Mistral)

In [ ]:
# creating a copy of the data
data_1 = data.copy()

**We are going to use an instruction-tuned Mistral model. Hence, the format of the input to the model varies from that of Llama.**

In [ ]:
#Defining the response funciton for Task 1.
def response_1(prompt,review):
    model_output = llm(
      f"""
      Q: {prompt}
      Review: {review}
      A:
      """,
      max_tokens=32,
      stop=["Q:", "\n"],
      temperature=0.01,
      echo=False,
    )

    temp_output = model_output["choices"][0]["text"]

    return temp_output

In [ ]:
# defining the instructions for the model
instruction_1 = """
    You are an AI analyzing restaurant reviews. Classify the sentiment of the provided review into the following categories:
    - Positive
    - Negative
    - Neutral
"""

In [ ]:
data_1['model_response'] = data_1['review_full'].apply(lambda x: response_1(instruction_1, x))

In [ ]:
data_1['model_response'].head()

In [ ]:
i = 2
print(data_1.loc[i, 'review_full'])

In [ ]:
print(data_1.loc[i, 'model_response'])

In [ ]:
def extract_sentiment(model_response):
    if 'positive' in model_response.lower():
        return 'Positive'
    elif 'negative' in model_response.lower():
        return 'Negative'
    elif 'neutral' in model_response.lower():
        return 'Neutral'

In [ ]:
# applying the function to the model response
data_1['sentiment'] = data_1['model_response'].apply(extract_sentiment)
data_1['sentiment'].head()

In [ ]:
data_1['sentiment'].value_counts()

In [ ]:
final_data_1 = data_1.drop(['model_response'], axis=1)
final_data_1.head()

## 2. Sentiment Analysis and Returning Structured Output (Llama)

In [ ]:
# creating a copy of the data
data_2 = data.copy()

NameError: name 'data' is not defined

In [ ]:
# defining the instructions for the model
instruction_2 = """
    You are an AI analyzing restaurant reviews. Classify the sentiment of the provided review into the following categories:
    - Positive
    - Negative
    - Neutral

    Format the output as a JSON object with a single key-value pair as shown below:
    {"sentiment": "your_sentiment_prediction"}
"""

In [ ]:
data_2['model_response'] = data_2['review_full'].apply(lambda x: generate_llama_response(instruction_2, x))


In [ ]:
data_2['model_response'].head()

In [ ]:
i = 2
print(data_2.loc[i, 'review_full'])

In [ ]:
print(data_2.loc[i, 'model_response'])

In [ ]:
# applying the function to the model response
data_2['model_response_parsed'] = data_2['model_response'].apply(extract_json_data)
data_2['model_response_parsed'].head()

In [ ]:
model_response_parsed_df_2 = pd.json_normalize(data_2['model_response_parsed'])
model_response_parsed_df_2.head()

In [ ]:
data_with_parsed_model_output_2 = pd.concat([data_2, model_response_parsed_df_2], axis=1)
data_with_parsed_model_output_2.head()

In [ ]:
final_data_2 = data_with_parsed_model_output_2.drop(['model_response','model_response_parsed'], axis=1)
final_data_2.head()

In [ ]:
final_data_2['sentiment'].value_counts()

## 3. Identifying Overall Sentiment and Sentiment of Aspects of the Experience (Llama)

In [ ]:
# creating a copy of the data
data_3 = data.copy()

In [ ]:
# defining the instructions for the model
instruction_3 = """
    You are an AI analyzing restaurant reviews. Classify the overall sentiment of the provided review into the following categories:
    - "Positive"
    - "Negative"
    - "Neutral"

    Once that is done, check for a mention of the following aspects in the review and classify the sentiment of each aspect as "Positive", "Negative", or "Neutral":
    1. "Food Quality"
    2. "Service"
    3. "Ambience"

    Output the overall sentiment and sentiment for each category in a JSON format with the following keys:
    {
        "Overall": "your_sentiment_prediction",
        "Food Quality": "your_sentiment_prediction",
        "Service": "your_sentiment_prediction",
        "Ambience": "your_sentiment_prediction"
    }

    In case one of the three aspects is not mentioned in the review, set "Not Applicable" (including quotes) for the corresponding JSON key value.
    Only return the JSON, do not return any other information.
"""

In [ ]:
data_3['model_response'] = data_3['review_full'].apply(lambda x: generate_llama_response(instruction_3, x))

In [ ]:
data_3['model_response'].head()

In [ ]:
i = 2
print(data_3.loc[i, 'review_full'])

In [ ]:
print(data_3.loc[i, 'model_response'])

In [ ]:
# applying the function to the model response
data_3['model_response_parsed'] = data_3['model_response'].apply(extract_json_data)
data_3['model_response_parsed'].head()

In [ ]:
model_response_parsed_df_3 = pd.json_normalize(data_3['model_response_parsed'])
model_response_parsed_df_3.head()

In [ ]:
data_with_parsed_model_output_3 = pd.concat([data_3, model_response_parsed_df_3], axis=1)
data_with_parsed_model_output_3.head()

In [ ]:
final_data_3 = data_with_parsed_model_output_3.drop(['model_response','model_response_parsed'], axis=1)
final_data_3.head()

In [ ]:
final_data_3['Overall'].value_counts()

In [ ]:
final_data_3['Food Quality'].value_counts()

In [ ]:
final_data_3['Service'].value_counts()

In [ ]:
final_data_3['Ambience'].value_counts()

## 3. Identifying Overall Sentiment and Sentiment of Aspects of the Experience (Mistral)

In [ ]:
# creating a copy of the data
data_3 = data.copy()

In [ ]:
def response_2(prompt,review,sentiment):
    model_output = llm(
      f"""
      Q: {prompt}
      review: {review}
      sentiment: {sentiment}
      A:
      """,
      max_tokens=64,
      stop=["Q:", "\n"],
      temperature=0.01,
      echo=False,
    )

    temp_output = model_output["choices"][0]["text"]
    final_output = temp_output[temp_output.index('{'):]

    return final_output

**Note:** We have already predicted the sentiment of the review. We can use this information while designing the prompt for this task. This way, it will reduce the computational complexity.

The sentiment is stored in the 'final_data_1' dataframe which is from the TASK 1.

In [ ]:
# defining the instructions for the model
instruction_3 = """
    You are provided a review and it's sentiment.

    Instructions:
    Classify the sentiment of each aspect as either of "Positive", "Negative", or "Neutral" only and not any other for the given review:
    1. "Food Quality"
    2. "Service"
    3. "Ambience"
    In case one of the three aspects is not mentioned in the review, return "Not Applicable" (including quotes) for the corresponding JSON key value.
    Return the output in the format {"Overall": given sentiment input,"Food Quality": "your_sentiment_prediction","Service": "your_sentiment_prediction","Ambience": "your_sentiment_prediction"}

"""

In [ ]:
data_3['model_response'] = final_data_1[['review_full','sentiment']].apply(lambda x: response_2(instruction_3, x[0],x[1]),axis=1)

In [ ]:
data_3['model_response'].values

In [ ]:
i = 2
print(data_3.loc[i, 'review_full'])

In [ ]:
print(data_3.loc[i, 'model_response'])

In [ ]:
# applying the function to the model response
data_3['model_response_parsed'] = data_3['model_response'].apply(extract_json_data)
data_3['model_response_parsed']

In [ ]:
model_response_parsed_df_3 = pd.json_normalize(data_3['model_response_parsed'])
model_response_parsed_df_3

In [ ]:
model_response_parsed_df_3 = model_response_parsed_df_3.apply(lambda x: x.astype(str).str.lower())

In [ ]:
data_with_parsed_model_output_3 = pd.concat([data_3, model_response_parsed_df_3], axis=1)
data_with_parsed_model_output_3.head()

In [ ]:
final_data_3 = data_with_parsed_model_output_3.drop(['model_response','model_response_parsed'], axis=1)
final_data_3.head()

In [ ]:
final_data_3['Overall'].value_counts()

In [ ]:
final_data_3['Food Quality'].value_counts()

**Note:** One of the sentiment is 'if not exceptional'. This is most likely positive.

In [ ]:
final_data_3['Service'].value_counts()

In [ ]:
final_data_3['Ambience'].value_counts()

## 4. Identifying Overall Sentiment, Sentiment of Aspects of the Experience, and the Liked/Disliked Features of the Different Aspects of the Experience (Llama)

In [1]:
# creating a copy of the data
data_4 = data.copy()

NameError: name 'data' is not defined

In [ ]:
# defining the instructions for the model
instruction_4 = """
    You are an AI tasked with analyzing restaurant reviews. Your goal is to classify the overall sentiment of the provided review into the following categories:
        - Positive
        - Negative
        - Neutral

    Subsequently, assess the sentiment of specific aspects mentioned in the review, namely:
        1. Food quality
        2. Service
        3. Ambience

    Further, identify liked and/or disliked features associated with each aspect in the review.

    Return the output in the specified JSON format, ensuring consistency and handling missing values appropriately:

    {
        "Overall": "your_sentiment_prediction",
        "Food Quality": "your_sentiment_prediction",
        "Service": "your_sentiment_prediction",
        "Ambience": "your_sentiment_prediction",
        "Food Quality Features": ["liked/disliked features"],
        "Service Features": ["liked/disliked features"],
        "Ambience Features": ["liked/disliked features"]
    }

    The sentiment prediction for Overall, Food Quality, Service, and Ambience should be one of "Positive", "Negative", or "Neutral" only.
    In case one of the three aspects is not mentioned in the review, set "Not Applicable" (including quotes) in the corresponding JSON key value for the sentiment.
    In case there are no liked/disliked features for a particular aspect, assign an empty list in the corresponding JSON key value for the aspect.
    Only return the JSON, do NOT return any other text or information.
"""

In [2]:
data_4['model_response'] = data_4['review_full'].apply(lambda x: generate_llama_response(instruction_4, x).replace('\n', ''))

NameError: name 'data_4' is not defined

In [3]:
i = 2
print(data_4.loc[i, 'review_full'])

NameError: name 'data_4' is not defined

In [4]:
print(data_4.loc[i, 'model_response'])

NameError: name 'data_4' is not defined

In [5]:
# applying the function to the model response
data_4['model_response_parsed'] = data_4['model_response'].apply(extract_json_data)
data_4['model_response_parsed'].head()

NameError: name 'data_4' is not defined

In [6]:
data_4[data_4.model_response_parsed == {}]

NameError: name 'data_4' is not defined

In [ ]:
print(data_4.loc[3, 'model_response'])

In [ ]:
upd_val_1 = {
    "Overall": "Positive",
    "Food Quality": "Positive",
    "Service": "Positive",
    "Ambience": "Not Applicable",
    "Food Quality Features": [],
    "Service Features": ["excellent service"],
    "Ambience Features": []
}

upd_val_2 = {
    "Overall": "Neutral",
    "Food Quality": "Neutral",
    "Service": "Neutral",
    "Ambience": "Not Applicable",
    "Food Quality Features": ["well prepared"],
    "Service Features": ["slow and inattentive"],
    "Ambience Features": ["interior is friendly", "not intimidating"]
}

upd_val_3 = {
    "Overall": "Neutral",
    "Food Quality": "Positive",
    "Service": "Negative",
    "Ambience": "Positive",
    "Food Quality Features": ["Some tasty, others average"],
    "Service Features": ["Attentive staff", "Slow service"],
    "Ambience Features": []
}

# defining the list of indices to update
idx_list = [3,6,7]
data_4.loc[idx_list, 'model_response_parsed'] = [upd_val_1, upd_val_2, upd_val_3]

In [7]:
model_response_parsed_df_4 = pd.json_normalize(data_4['model_response_parsed'])
model_response_parsed_df_4.head()

NameError: name 'pd' is not defined

In [ ]:
model_response_parsed_df_4 = pd.json_normalize(data_4['model_response_parsed'])
model_response_parsed_df_4.head()

In [ ]:
data_with_parsed_model_output_4 = pd.concat([data_4, model_response_parsed_df_4], axis=1)
data_with_parsed_model_output_4.head()

In [ ]:
final_data_4 = data_with_parsed_model_output_4.drop(['model_response','model_response_parsed'], axis=1)
final_data_4.head()

In [ ]:
final_data_4['Overall'].value_counts()

In [ ]:
final_data_4['Food Quality'].value_counts()

In [ ]:
final_data_4['Service'].value_counts()

In [ ]:
final_data_4['Ambience'].value_counts()final_data_4['Ambience'].value_counts()

## 5. Identifying Overall Sentiment, Sentiment of Aspects of the Experience, Liked/Disliked Features of the Different Aspects of the Experience, and Sharing a Response (Llama)

In [ ]:
# creating a copy of the data
data_5 = data.copy()

In [ ]:
# defining the instructions for the model
instruction_5 = """
    You are an AI analyzing restaurant reviews. Classify the overall sentiment of the provided review into the following categories:
    - "Positive"
    - "Negative"
    - "Neutral"

    Once that is done, check for a mention of the following aspects in the review and clasify the sentiment of each aspect as positive, negative, or neutral:
    1. Food quality
    2. Service
    3. Ambience

    Once that is done, look for liked and/or disliked features mentioned against each of the above aspects in the review and extract them.

    Finally, draft a response for the customer based on the review. Start out with a thank you note and then add on to it as per the following:
    1. If the review is positive, mention that it would be great to have them again
    2. If the review is neutral, ask them for what the restaurant could have done better
    3. If the review is negative, apologive for the inconvenience and mention that we'll be looking into the points raised

    Return the output in the specified JSON format, ensuring consistency and handling missing values appropriately Ensure that all values in the JSON are formatted as strings, and each element within the lists should be enclosed in double quotes:

    {
        "Overall": "your_sentiment_prediction",
        "Food Quality": "your_sentiment_prediction",
        "Service": "your_sentiment_prediction",
        "Ambience": "your_sentiment_prediction",
        "Food Quality Features": ["liked/disliked features"],
        "Service Features": ["liked/disliked features"],
        "Ambience Features": ["liked/disliked features"],
        "Response": "your_response_to_the_customer_review",
    }

    The sentiment prediction for Overall, Food Quality, Service, and Ambience should be one of "Positive", "Negative", or "Neutral" only.
    In case one of the three aspects is not mentioned in the review, set "Not Applicable" (including quotes) in the corresponding JSON key value for the sentiment.
    In case there are no liked/disliked features for a particular aspect, assign an empty list in the corresponding JSON key value for the aspect.
    Be polite and empathetic in the response to the customer review.
    Only return the JSON, do NOT return any other text or information.
"""

In [ ]:
data_5['model_response'] = data_5['review_full'].apply(lambda x: generate_llama_response(instruction_5, x))

In [ ]:
i = 2
print(data_5.loc[i, 'review_full'])

In [ ]:
print(data_5.loc[i, 'model_response'])

In [ ]:
# applying the function to the model response
data_5['model_response_parsed'] = data_5['model_response'].apply(extract_json_data)
data_5['model_response_parsed'].head()

In [ ]:
model_response_parsed_df_5 = pd.json_normalize(data_5['model_response_parsed'])
model_response_parsed_df_5.head()

In [ ]:
model_response_parsed_df_5['Response'][0]

In [ ]:
data_with_parsed_model_output_5 = pd.concat([data_5, model_response_parsed_df_5], axis=1)
data_with_parsed_model_output_5.head()

In [ ]:
final_data_5 = data_with_parsed_model_output_5.drop(['model_response','model_response_parsed'], axis=1)
final_data_5.head()

In [ ]:
final_data_5['Overall'].value_counts()

In [ ]:
final_data_5['Food Quality'].value_counts()

In [ ]:
final_data_5['Service'].value_counts()

In [ ]:
final_data_5['Ambience'].value_counts()

final_data_5['Service'].value_counts()
- We used an LLM to do multiple tasks, one stage at a time
    1. We first identified the overall sentiment of the review using the LLM
    2. We then identified the overall sentiment of the review and got the output in a structured format from the LLM for ease-of-access
    3. Next, we identified the overall sentiment of the review as well as sentiment of specific aspects of the experience
    4. Next, in addition to the overall sentiment of the review as well as sentiment of specific aspects of the experience, we also identified the liked/disliked features of the different aspects of the experience
    5. Finally, in addition to all the above, we also got a response we can share with the customer based on their review

- One can manually label the data (overall sentiment and sentiments of different aspects) and then compare the model's output with the same to get a quantitative measure of the models performance.

- To try and improve the model performance, one can try the following:
    1. Update the prompt
    2. Update the model parameters (`temparature`, `top_p`, ...)

